# Fashion-MNIST TCN


RNN worflow based on the work of [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/) and [Sungjoon](https://github.com/sjchoi86/tensorflow-101/blob/master/notebooks/rnn_mnist_simple.ipynb)

Good resource: [How to use Dataset in TensorFlow](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428)

### Temporal Convolutional Networks Overview

![TCNs](https://cdn-images-1.medium.com/max/1000/1*1cK-UEWHGaZLM-4ITCeqdQ.png)

## System Information

In [0]:
!pip install watermark

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
%load_ext watermark
%watermark -v -m -p tensorflow,numpy -g

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


CPython 3.5.2
IPython 6.2.1

tensorflow 1.5.0
numpy 1.14.0

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.10.0-32-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : 5468040afa72013f062c448adadef29b54276f44


In [0]:
from pathlib import Path
import random 
from datetime import datetime

import tensorflow as tf
import numpy as np
import pandas as pd

## Prepare FMNIST

In [0]:
df_train = pd.read_csv("../data/fashion-mnist_train.csv")
df_test = pd.read_csv("../data/fashion-mnist_test.csv")
print(df_train.shape, df_test.shape)

(60000, 785) (10000, 785)


In [0]:
print(df_train.columns)
print(df_test.columns)

Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)
Index(['label', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779', 'pixel780',
       'pixel781', 'pixel782', 'pixel783', 'pixel784'],
      dtype='object', length=785)


In [0]:
# Train/Validation Split
idx = np.arange(60000)
np.random.shuffle(idx)
print(idx[:5])
df_val = df_train.iloc[idx[:10000]]
df_train = df_train.iloc[idx[10000:]]
print(df_val.shape, df_train.shape)

[42394 37741 33781 18995 34876]
(10000, 785) (50000, 785)


## TCN

In [0]:
class CausalConv1D(tf.layers.Conv1D):
    def __init__(self, filters,
               kernel_size,
               strides=1,
               dilation_rate=1,
               activation=None,
               use_bias=True,
               kernel_initializer=None,
               bias_initializer=tf.zeros_initializer(),
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               trainable=True,
               name=None,
               **kwargs):
        super(CausalConv1D, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding='valid',
            data_format='channels_last',
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=name, **kwargs
        )
       
    def call(self, inputs):
        padding = (self.kernel_size[0] - 1) * self.dilation_rate[0]
        inputs = tf.pad(inputs, tf.constant([(0, 0,), (1, 0), (0, 0)]) * padding)
        return super(CausalConv1D, self).call(inputs)

In [0]:
class TemporalBlock(tf.layers.Layer):
    def __init__(self, n_outputs, kernel_size, strides, dilation_rate, dropout=0.2, 
                 trainable=True, name=None, dtype=None, batch_size=32,
                 activity_regularizer=None, **kwargs):
        super(TemporalBlock, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )        
        self.dropout = dropout
        self.batch_size = tf.cast(batch_size, tf.int32)
        self.n_outputs = n_outputs
        self.conv1 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv1")
        self.conv2 = CausalConv1D(
            n_outputs, kernel_size, strides=strides, 
            dilation_rate=dilation_rate, activation=tf.nn.relu, 
            name="conv2")
        self.down_sample = None

    
    def build(self, input_shape):
        channel_dim = 2
        self.dropout1 = tf.layers.Dropout(self.dropout, [self.batch_size, tf.constant(1), tf.constant(self.n_outputs)])
        self.dropout2 = tf.layers.Dropout(self.dropout, [self.batch_size, tf.constant(1), tf.constant(self.n_outputs)])
        if input_shape[channel_dim] != self.n_outputs:
            # self.down_sample = tf.layers.Conv1D(
            #     self.n_outputs, kernel_size=1, 
            #     activation=None, data_format="channels_last", padding="valid")
            self.down_sample = tf.layers.Dense(self.n_outputs, activation=None)
        self.built = True
    
    def call(self, inputs, training=True):
        x = self.conv1(inputs)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = tf.contrib.layers.layer_norm(x)
        x = self.dropout2(x, training=training)
        if self.down_sample is not None:
            inputs = self.down_sample(inputs)
        return tf.nn.relu(x + inputs)

In [0]:
class TemporalConvNet(tf.layers.Layer):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2,
                 trainable=True, name=None, dtype=None, batch_size=32,
                 activity_regularizer=None, **kwargs):
        super(TemporalConvNet, self).__init__(
            trainable=trainable, dtype=dtype,
            activity_regularizer=activity_regularizer,
            name=name, **kwargs
        )
        self.layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            out_channels = num_channels[i]
            self.layers.append(
                TemporalBlock(
                    out_channels, kernel_size, strides=1, 
                    dilation_rate=dilation_size, batch_size=batch_size,
                    dropout=dropout, name="tblock_{}".format(i))
            )
    
    def call(self, inputs, training=True):
        outputs = inputs
        for layer in self.layers:
            outputs = layer(outputs, training=training)
        return outputs

## Build the Model

In [0]:
# Training Parameters
learning_rate = 0.001
batch_size = 32
display_step = 500
print("Total batches per epoch:", np.ceil(df_train.shape[0] / batch_size))
training_steps = 10000

# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 28 * 28 # timesteps
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.05
kernel_size = 8
levels = 6
nhid = 24 # hidden layer num of features

Total batches per epoch: 1563.0


In [0]:
def process_batch(batch_x, batch_y):
    return tf.expand_dims(batch_x, -1), tf.one_hot(batch_y, num_classes)

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    
    with tf.variable_scope("datasets"):
        training_batch_size = tf.placeholder(tf.int64) # tf.constant(32, dtype="int64")
        inference_batch_size = tf.placeholder(tf.int64) # tf.constant(500, dtype="int64")
        is_training = tf.placeholder("bool")

        fminst_ds_train = tf.data.Dataset.from_tensor_slices(
            (df_train.iloc[:, 1:].astype("float32") / 255, df_train.iloc[:, 0].astype("int32"))
        ).shuffle(50000, reshuffle_each_iteration=True).repeat().batch(training_batch_size).map(process_batch)
        fminst_ds_val = tf.data.Dataset.from_tensor_slices(
            (df_val.iloc[:, 1:].astype("float32") / 255, df_val.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)
        fminst_ds_test = tf.data.Dataset.from_tensor_slices((
            df_test.iloc[:, 1:].astype("float32") / 255, df_test.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)

        handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            handle, fminst_ds_train.output_types, fminst_ds_train.output_shapes)

        train_iterator = fminst_ds_train.make_initializable_iterator()
        val_iterator = fminst_ds_val.make_initializable_iterator()
        test_iterator = fminst_ds_test.make_initializable_iterator() 
    
    X_0, Y = iterator.get_next()
    X = tf.reshape(X_0, (-1, timesteps, num_input))

    # Define weights
    logits = tf.layers.dense(
        TemporalConvNet(
            [nhid] * levels, kernel_size, dropout, 
            batch_size=tf.shape(X)[0])(
            X, training=is_training)[:, -1, :],
        num_classes, activation=None, 
        kernel_initializer=tf.orthogonal_initializer()
    )
    prediction = tf.nn.softmax(logits)
   
    with tf.name_scope("loss"):
        # Define loss and optimizer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=Y))
        ema_loss = tf.get_variable("ema_loss", shape=[], dtype=tf.float32, trainable=False, initializer=tf.constant_initializer(2.5))
        ema_update = ema_loss.assign(ema_loss * 0.99 + loss * 0.01)
    tf.summary.scalar('Loss', ema_loss)
    
    with tf.variable_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        with tf.control_dependencies([ema_update]):
            train_op = optimizer.minimize(loss)

    with tf.variable_scope("inference"):
        # Evaluate model (with test logits, for dropout to be disabled)
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    merged_summary_op = tf.summary.merge_all()
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))

All parameters: 156129.0
Trainable parameters: 52042


### Training and Evaluating

In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8

train_writer = tf.summary.FileWriter("logs/fminst_tcn/%s/train" % datetime.now().strftime("%Y%m%d_%H%M"), graph)
val_writer = tf.summary.FileWriter("logs/fminst_tcn/%s/val" % datetime.now().strftime("%Y%m%d_%H%M"))

with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer    
    sess.run(init) 
    sess.run([train_iterator.initializer, val_iterator.initializer, test_iterator.initializer],
             feed_dict={training_batch_size: batch_size, inference_batch_size: 500})
    train_handle, val_handle, test_handle = sess.run(
        [train_iterator.string_handle(), val_iterator.string_handle(), test_iterator.string_handle()])
    for step in range(1, training_steps+1):
        sess.run([train_op], feed_dict={handle: train_handle, is_training: True})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss_local, acc, summary = sess.run(
                [loss, accuracy, merged_summary_op], 
                feed_dict={handle: train_handle, is_training: False})
            train_writer.add_summary(summary, global_step=step)
            train_writer.flush()            
            val_acc, val_loss = [], []
            for _ in range(20):
                tmp = sess.run(
                    [accuracy, loss], 
                    feed_dict={handle: val_handle, is_training: False})
                val_acc.append(tmp[0])
                val_loss.append(tmp[1])
            summary = tf.Summary()
            val_loss = np.mean(val_loss)            
            val_acc = np.mean(val_acc)            
            summary.value.add(tag='Loss', simple_value=val_loss)
            val_writer.add_summary(summary, global_step=step)
            val_writer.flush() 
            print("Step " + str(step) + ", Train Batch Loss= " + \
                  "{:.4f}".format(loss_local) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Val Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt")
                print("Model saved in path: %s" % save_path)  
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle, is_training: False}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))                
    print("Optimization Finished!")
    
train_writer.close()
val_writer.close()

Step 1, Train Batch Loss= 3.0748, Training Accuracy= 0.125, Val Accuracy= 0.138
Step 500, Train Batch Loss= 0.4752, Training Accuracy= 0.812, Val Accuracy= 0.777
Step 1000, Train Batch Loss= 0.2384, Training Accuracy= 0.969, Val Accuracy= 0.826
Model saved in path: /tmp/model.ckpt
Step 1500, Train Batch Loss= 0.5771, Training Accuracy= 0.750, Val Accuracy= 0.835
Model saved in path: /tmp/model.ckpt
Step 2000, Train Batch Loss= 0.5490, Training Accuracy= 0.781, Val Accuracy= 0.851
Model saved in path: /tmp/model.ckpt
Step 2500, Train Batch Loss= 0.1811, Training Accuracy= 0.969, Val Accuracy= 0.854
Model saved in path: /tmp/model.ckpt
Step 3000, Train Batch Loss= 0.3573, Training Accuracy= 0.844, Val Accuracy= 0.862
Model saved in path: /tmp/model.ckpt
Step 3500, Train Batch Loss= 0.0655, Training Accuracy= 1.000, Val Accuracy= 0.870
Model saved in path: /tmp/model.ckpt
Step 4000, Train Batch Loss= 0.4601, Training Accuracy= 0.781, Val Accuracy= 0.854
Step 4500, Train Batch Loss= 0.3019

In [0]:
with tf.Session(graph=graph, config=config) as sess:
    saver.restore(sess, '/tmp/model.ckpt')
    sess.run(test_iterator.initializer,
             feed_dict={inference_batch_size: 500})
    test_handle = sess.run(test_iterator.string_handle())    
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle, is_training: False}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Test Accuracy= 0.895


## Permute

In [0]:
# Training Parameters
training_steps = 15000

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    
    with tf.variable_scope("datasets"):
        training_batch_size = tf.placeholder(tf.int64) # tf.constant(32, dtype="int64")
        inference_batch_size = tf.placeholder(tf.int64) # tf.constant(500, dtype="int64")
        is_training = tf.placeholder("bool")

        fminst_ds_train = tf.data.Dataset.from_tensor_slices(
            (df_train.iloc[:, 1:].astype("float32") / 255, df_train.iloc[:, 0].astype("int32"))
        ).shuffle(50000, reshuffle_each_iteration=True).repeat().batch(training_batch_size).map(process_batch)
        fminst_ds_val = tf.data.Dataset.from_tensor_slices(
            (df_val.iloc[:, 1:].astype("float32") / 255, df_val.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)
        fminst_ds_test = tf.data.Dataset.from_tensor_slices((
            df_test.iloc[:, 1:].astype("float32") / 255, df_test.iloc[:, 0].astype("int32"))
        ).repeat().batch(inference_batch_size).map(process_batch)

        handle = tf.placeholder(tf.string, shape=[])
        iterator = tf.data.Iterator.from_string_handle(
            handle, fminst_ds_train.output_types, fminst_ds_train.output_shapes)

        train_iterator = fminst_ds_train.make_initializable_iterator()
        val_iterator = fminst_ds_val.make_initializable_iterator()
        test_iterator = fminst_ds_test.make_initializable_iterator() 
    
    X_0, Y = iterator.get_next()
    
    # Permute the time step
    np.random.seed(100)
    permute = np.random.permutation(784)
    X = tf.gather(
        tf.reshape(X_0, (-1, timesteps, num_input)), 
        permute, axis=1)

    # Define weights
    logits = tf.layers.dense(
        TemporalConvNet([nhid] * levels, kernel_size, dropout)(
            X, training=is_training)[:, -1, :],
        num_classes, activation=None, 
        kernel_initializer=tf.orthogonal_initializer()
    )
    prediction = tf.nn.softmax(logits)
   
    with tf.name_scope("loss"):
        # Define loss and optimizer
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=Y))
        ema_loss = tf.get_variable("ema_loss", shape=[], dtype=tf.float32, trainable=False, initializer=tf.constant_initializer(2.5))
        ema_update = ema_loss.assign(ema_loss * 0.99 + loss * 0.01)
    tf.summary.scalar('Loss', ema_loss)
    
    with tf.variable_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        with tf.control_dependencies([ema_update]):
            train_op = optimizer.minimize(loss)

    with tf.variable_scope("inference"):
        # Evaluate model (with test logits, for dropout to be disabled)
        correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    merged_summary_op = tf.summary.merge_all()
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))

All parameters: 156129.0
Trainable parameters: 52042


In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8

train_writer = tf.summary.FileWriter("logs/fminst_tcn/%s_permute/train" % datetime.now().strftime("%Y%m%d_%H%M"), graph)
val_writer = tf.summary.FileWriter("logs/fminst_tcn/%s_permute/val" % datetime.now().strftime("%Y%m%d_%H%M"))

with tf.Session(graph=graph, config=config) as sess:
    sess.run(init) 
    sess.run([train_iterator.initializer, val_iterator.initializer, test_iterator.initializer],
             feed_dict={training_batch_size: batch_size, inference_batch_size: 500})
    train_handle, val_handle, test_handle = sess.run(
        [train_iterator.string_handle(), val_iterator.string_handle(), test_iterator.string_handle()])
    # Run the initializer
    for step in range(1, training_steps+1):
        sess.run([train_op], feed_dict={handle: train_handle, is_training: True})
        
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss_local, acc, summary = sess.run(
                [loss, accuracy, merged_summary_op], 
                feed_dict={handle: train_handle, is_training: False})
            train_writer.add_summary(summary, global_step=step)
            train_writer.flush()            
            val_acc, val_loss = [], []
            for _ in range(20):
                tmp = sess.run(
                    [accuracy, loss], 
                    feed_dict={handle: val_handle, is_training: False})
                val_acc.append(tmp[0])
                val_loss.append(tmp[1])
            summary = tf.Summary()
            val_loss = np.mean(val_loss)            
            val_acc = np.mean(val_acc)            
            summary.value.add(tag='Loss', simple_value=val_loss)
            val_writer.add_summary(summary, global_step=step)
            val_writer.flush() 
            print("Step " + str(step) + ", Train Batch Loss= " + \
                  "{:.4f}".format(loss_local) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Val Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt")
                print("Model saved in path: %s" % save_path)  
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle, is_training: False}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))                
    print("Optimization Finished!")
    
train_writer.close()
val_writer.close()

Step 1, Train Batch Loss= 3.6829, Training Accuracy= 0.094, Val Accuracy= 0.151
Step 500, Train Batch Loss= 0.4500, Training Accuracy= 0.781, Val Accuracy= 0.808
Model saved in path: /tmp/model.ckpt
Step 1000, Train Batch Loss= 0.3328, Training Accuracy= 0.875, Val Accuracy= 0.826
Model saved in path: /tmp/model.ckpt
Step 1500, Train Batch Loss= 0.5785, Training Accuracy= 0.750, Val Accuracy= 0.836
Model saved in path: /tmp/model.ckpt
Step 2000, Train Batch Loss= 0.7037, Training Accuracy= 0.719, Val Accuracy= 0.842
Model saved in path: /tmp/model.ckpt
Step 2500, Train Batch Loss= 0.2977, Training Accuracy= 0.875, Val Accuracy= 0.852
Model saved in path: /tmp/model.ckpt
Step 3000, Train Batch Loss= 0.5172, Training Accuracy= 0.812, Val Accuracy= 0.856
Model saved in path: /tmp/model.ckpt
Step 3500, Train Batch Loss= 0.0847, Training Accuracy= 1.000, Val Accuracy= 0.862
Model saved in path: /tmp/model.ckpt
Step 4000, Train Batch Loss= 0.4534, Training Accuracy= 0.812, Val Accuracy= 0.85

In [0]:
with tf.Session(graph=graph, config=config) as sess:
    saver.restore(sess, '/tmp/model.ckpt')
    sess.run(test_iterator.initializer,
             feed_dict={inference_batch_size: 500})
    test_handle = sess.run(test_iterator.string_handle())    
    test_acc = []
    for _ in range(20):
        test_acc.append(sess.run(accuracy, feed_dict={handle: test_handle, is_training: False}))
    test_acc = np.mean(test_acc)
    print("Test Accuracy= {:.3f}".format(test_acc))

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Test Accuracy= 0.881
